I am building a model to predict When can the Toronto Police expect a surge of crime in Toronto (Day of week, Time of Day and Month ) based on the Toronto police crime data set. 
To do this, I would be exploring 3 models 

1. Time Series Forecasting Models(Xgboost): I am chosing this model because crime occurance often show temporal dependencies which is that they current trends and this model is good at detecting such patterns and seasonality.

2. Ensemble Learning Models: This model helps to capture non-linear relationships among features for both categorical and numeric data.  

3. Bayesian models: Incorporates uncertainty by using probabilty which is good when dealing with limited data 

First model we are building is the time series forcasting model. 
1. The first thing we need is the trend of crime occurance 
2. The second thing is seasonality. This is the repeated pattern of data over a set period in time. Eg the way crime occurs during maybe the holiday season, or weekends, or summer periods
3. Cycle is the non-seasonal patterns in the data. This could be occurance during maybe the close of down of the pandemic or economic downturn 
4. Variations are unpredictable up and down that can not be explained by the other variables above 

Models examples
ARIMA : Auto regressive integrated moving average : This looks at how past values affect future values through trends, seasonality and cycles while removing the noise (Variations that dont make sense) 

Expoential smoothing : This is used for data that has no trend, seasonality or cycle but only has variations. It works by giving more weight to recent value and less weight to previous values

Implementation 


In [1]:
pip install xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np


In [3]:
crime_data = pd.read_csv('https://raw.githubusercontent.com/Pelumioluwa/MBAN6110Class/main/Data/Toronto%20Police%20Shooting%20and%20Firearm%20Discharge.csv')
crime_data

,X,Y,OBJECTID,EVENT_UNIQUE_ID,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DOW,OCC_DOY,OCC_DAY,...,OCC_TIME_RANGE,DIVISION,DEATH,INJURIES,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-79.234955,43.781528,1,GO-2004397105,2004/05/15 04:00:00+00,2004,May,Saturday,136,15,...,Night,D41,0,0,142,Woburn North,137,Woburn (137),-79.234955,43.781528
1,-79.618218,43.733547,2,GO-2004600109,2004/07/26 04:00:00+00,2004,July,Monday,208,26,...,Evening,D23,0,0,1,West Humber-Clairville,1,West Humber-Clairville (1),-79.618218,43.733547
2,-79.518759,43.769157,3,GO-2004311851,2004/05/14 04:00:00+00,2004,May,Friday,135,14,...,Afternoon,D31,1,1,24,Black Creek,24,Black Creek (24),-79.518759,43.769157
3,-79.378870,43.660665,4,GO-2004736004,2004/12/19 05:00:00+00,2004,December,Sunday,354,19,...,Night,D51,1,0,168,Downtown Yonge East,75,Church-Yonge Corridor (75),-79.378870,43.660665
4,-79.212435,43.812075,5,GO-2004303455,2004/04/12 04:00:00+00,2004,April,Monday,103,12,...,Evening,D42,0,0,146,Malvern East,132,Malvern (132),-79.212435,43.812075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,-79.362941,43.771764,5703,GO-20221038418,2022/06/02 04:00:00+00,2022,June,Thursday,153,2,...,Night,D33,0,1,53,Henry Farm,53,Henry Farm (53),-79.362941,43.771764
5703,-79.402702,43.643920,5704,GO-20221362949,2022/07/17 04:00:00+00,2022,July,Sunday,198,17,...,Night,D14,1,1,164,Wellington Place,77,Waterfront Communities-The Island (77),-79.402702,43.643920
5704,-79.196217,43.786621,5705,GO-2022336423,2022/02/21 05:00:00+00,2022,February,Monday,52,21,...,Night,D43,0,0,135,Morningside,135,Morningside (135),-79.196217,43.786621
5705,-79.443993,43.658083,5706,GO-20221765442,2022/09/11 04:00:00+00,2022,September,Sunday,254,11,...,Night,D11,0,2,83,Dufferin Grove,83,Dufferin Grove (83),-79.443993,43.658083


In [111]:
crime_data.shape

(5707, 21)

In [112]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5707 entries, 0 to 5706
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  5707 non-null   float64
 1   Y                  5707 non-null   float64
 2   OBJECTID           5707 non-null   int64  
 3   EVENT_UNIQUE_ID    5707 non-null   object 
 4   OCC_DATE           5707 non-null   object 
 5   OCC_YEAR           5707 non-null   int64  
 6   OCC_MONTH          5707 non-null   object 
 7   OCC_DOW            5707 non-null   object 
 8   OCC_DOY            5707 non-null   int64  
 9   OCC_DAY            5707 non-null   int64  
 10  OCC_HOUR           5707 non-null   int64  
 11  OCC_TIME_RANGE     5707 non-null   object 
 12  DIVISION           5707 non-null   object 
 13  DEATH              5707 non-null   int64  
 14  INJURIES           5707 non-null   int64  
 15  HOOD_158           5707 non-null   object 
 16  NEIGHBOURHOOD_158  5707 

In [113]:
#convert the occurance date to datetime 
#crime_data['OCC_DATE'] = pd.to_datetime(crime_data['OCC_DATE'])
#crime_data.info()

In [4]:
crime_data.columns

Index(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR',
       'OCC_MONTH', 'OCC_DOW', 'OCC_DOY', 'OCC_DAY', 'OCC_HOUR',
       'OCC_TIME_RANGE', 'DIVISION', 'DEATH', 'INJURIES', 'HOOD_158',
       'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84',
       'LAT_WGS84'],
      dtype='object')

Build a pipeline to convert the dataframe 
1. drop unnecessary columns not needed in the - X', 'Y', 'OBJECTID','DIVISION', 'DEATH', 'INJURIES', 'HOOD_158','NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'
2. Convert day of the week, month, day of the week, occurance time range to dummy data and one encode it 
3. Scale the numerical variables (day of the year,day of the month, hour of the day)
4. Make date time the index



In [5]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_MONTH','OCC_TIME_RANGE']
numerical_features = ['OCC_DOY','OCC_DAY','OCC_HOUR']
label = ['OCC_DOW']


Predict the probability that each day would have a crime , the period of the day and the month and the time of the day)

In [6]:
crime_data[label]

,OCC_DOW
0,Saturday
1,Monday
2,Friday
3,Sunday
4,Monday
...,...
5702,Thursday
5703,Sunday
5704,Monday
5705,Sunday


In [7]:
label_encoder_trans = LabelEncoder()
#crime_data['OCC_TIME_RANGE'] = label_encoder_trans.fit_transform(crime_data['OCC_TIME_RANGE'])
#crime_data['OCC_MONTH'] = label_encoder_trans.fit_transform(crime_data['OCC_MONTH'])
#crime_data['OCC_HOUR'] = label_encoder_trans.fit_transform(crime_data['OCC_HOUR'])
#crime_data['OCC_DOW'] = label_encoder_trans.fit_transform(crime_data['OCC_DOW'])
#target = crime_data['OCC_DOW']

In [8]:
crime_data[label]

,OCC_DOW
0,Saturday
1,Monday
2,Friday
3,Sunday
4,Monday
...,...
5702,Thursday
5703,Sunday
5704,Monday
5705,Sunday


In [14]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[label],
                                                    test_size = 0.3, random_state=123)

In [10]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])
label_encoder_trans = Pipeline(steps=[('label_encoder', LabelEncoder())])

In [18]:
#building processor
preprocessor = ColumnTransformer(transformers=[('categorical', categorical_transformer, categorical_features),
                                               ('numerical', numerical_transformer, numerical_features),
                                               ('label', label_encoder_trans, label)])
#preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),('numerical',numerical_transformer,numerical_features) ])

In [122]:
#define transformers and encoder
#categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder())])
#numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])
#label_encoder_trans = Pipeline(steps=[('label_encoder', LabelEncoder())])


In [123]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5707 entries, 0 to 5706
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  5707 non-null   float64
 1   Y                  5707 non-null   float64
 2   OBJECTID           5707 non-null   int64  
 3   EVENT_UNIQUE_ID    5707 non-null   object 
 4   OCC_DATE           5707 non-null   object 
 5   OCC_YEAR           5707 non-null   int64  
 6   OCC_MONTH          5707 non-null   object 
 7   OCC_DOW            5707 non-null   int64  
 8   OCC_DOY            5707 non-null   int64  
 9   OCC_DAY            5707 non-null   int64  
 10  OCC_HOUR           5707 non-null   int64  
 11  OCC_TIME_RANGE     5707 non-null   object 
 12  DIVISION           5707 non-null   object 
 13  DEATH              5707 non-null   int64  
 14  INJURIES           5707 non-null   int64  
 15  HOOD_158           5707 non-null   object 
 16  NEIGHBOURHOOD_158  5707 

In [19]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [20]:
random_forest.fit(X_train, y_train)

ValueError: A given column is not a column of the dataframe

In [126]:
# Predict the probabilities of crime occurrence for each day of the week in the test set
day_probabilities = random_forest.predict_proba(X_test)

In [127]:
# Display the probabilities for each day of the week in the test set
day_names = label_encoder_trans.inverse_transform(random_forest.classes_)
probabilities_df = pd.DataFrame(day_probabilities, columns=day_names)
print(probabilities_df)

        Friday    Monday  Saturday    Sunday  Thursday  Tuesday  Wednesday
0     0.050000  0.015000  0.100000  0.580000  0.140000    0.000   0.115000
1     0.000000  0.030000  0.120000  0.080000  0.560000    0.190   0.020000
2     0.459000  0.020000  0.010000  0.010000  0.050000    0.451   0.000000
3     0.120000  0.370000  0.225000  0.200000  0.000000    0.075   0.010000
4     0.107500  0.000000  0.092500  0.030000  0.020000    0.040   0.710000
...        ...       ...       ...       ...       ...      ...        ...
1708  0.040000  0.160167  0.000000  0.013667  0.772833    0.010   0.003333
1709  0.350714  0.000000  0.333214  0.000000  0.276071    0.000   0.040000
1710  0.055000  0.635000  0.000000  0.220000  0.010000    0.070   0.010000
1711  0.603333  0.102667  0.224000  0.050000  0.020000    0.000   0.000000
1712  0.000000  0.090000  0.165833  0.680000  0.000000    0.010   0.054167

[1713 rows x 7 columns]


Further to the predictions, I want to evaluate the accuracy of my model 

In [128]:
#import evaluation metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score


In [129]:
predicted_classes = day_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to day names
predicted_day_names = label_encoder_trans.inverse_transform(predicted_classes)
print(predicted_day_names)
print(predicted_classes)


['Sunday' 'Thursday' 'Friday' ... 'Monday' 'Friday' 'Sunday']
[3 4 0 ... 1 0 3]


In [130]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print("Accuracy:", accuracy)

Accuracy: 0.18914185639229422


In [131]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[41 35 42 38 31 27 25]
 [32 37 32 35 26 28 40]
 [36 36 67 57 23 38 33]
 [35 53 61 74 31 28 35]
 [30 30 35 32 26 23 26]
 [28 23 28 38 28 42 37]
 [23 33 41 36 19 22 37]]


In [132]:
# Calculate classification report (includes precision, recall, and F1-score for each class)
classification_rep = classification_report(y_test, predicted_classes, target_names=label_encoder_trans.classes_)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

      Friday       0.18      0.17      0.18       239
      Monday       0.15      0.16      0.16       230
    Saturday       0.22      0.23      0.22       290
      Sunday       0.24      0.23      0.24       317
    Thursday       0.14      0.13      0.13       202
     Tuesday       0.20      0.19      0.19       224
   Wednesday       0.16      0.18      0.17       211

    accuracy                           0.19      1713
   macro avg       0.18      0.18      0.18      1713
weighted avg       0.19      0.19      0.19      1713



In [133]:
# Calculate AUC-ROC (requires one-hot encoded y_test and day_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
auc_roc = roc_auc_score(y_test_one_hot, day_probabilities, multi_class='ovr')
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.5558068566867956


In [134]:
#get the day with the high mean of probability 
probabilities_df.describe(include='all')

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.138135,0.141227,0.171014,0.170812,0.120976,0.127126,0.130711
std,0.185236,0.193201,0.207591,0.215437,0.169832,0.181800,0.184511
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.010000,0.010000,0.020000,0.010000,0.010000,0.010000,0.010000
50%,0.050000,0.050000,0.080000,0.070000,0.042500,0.041667,0.045000
75%,0.200000,0.193345,0.252333,0.260000,0.166000,0.160000,0.171000
max,0.940000,0.904333,0.990000,0.973333,0.890000,0.948333,0.950214


In [135]:
#get the day with the highest sum of probability 
day_sum_prob = probabilities_df.sum()
print(day_sum_prob)
df_day_sum_prob = pd.DataFrame(day_sum_prob)


Friday       236.625262
Monday       241.921313
Saturday     292.946693
Sunday       292.600743
Thursday     207.232278
Tuesday      217.766508
Wednesday    223.907202
dtype: float64


In [136]:
df_day_sum_prob.reset_index()

,index,0
0,Friday,236.625262
1,Monday,241.921313
2,Saturday,292.946693
3,Sunday,292.600743
4,Thursday,207.232278
5,Tuesday,217.766508
6,Wednesday,223.907202


In [170]:
#renaming the column probabilities 
df_day_sum_prob = df_day_sum_prob.rename(columns={0: 'Cum_prob'})


In [171]:
df_day_sum_prob

,Cum_prob
Friday,236.625262
Monday,241.921313
Saturday,292.946693
Sunday,292.600743
Thursday,207.232278
Tuesday,217.766508
Wednesday,223.907202


In [137]:
#scale it down with min_max
min_max_scaler = MinMaxScaler()
scaled_prob = min_max_scaler.fit_transform(df_day_sum_prob)
scaled_df = pd.DataFrame(scaled_prob,columns=df_day_sum_prob.columns)
scaled_df


,0
0,0.342918
1,0.404705
2,1.000000
3,0.995964
4,0.000000
5,0.122899
6,0.194540


From the above, I can deduce that crime would most likely happen on a saturday and then a sunday. Therefore the weekends are the most vunerable days . 

In [138]:
#label the month 
crime_data['OCC_MONTH'] = label_encoder_trans.fit_transform(crime_data['OCC_MONTH'])
target = crime_data['OCC_MONTH']

In [139]:
crime_data['OCC_MONTH']

0        8
1        5
2        8
3        2
4        0
        ..
5702     6
5703     5
5704     3
5705    11
5706     0
Name: OCC_MONTH, Length: 5707, dtype: int64

In [140]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_DOW','OCC_TIME_RANGE']
numerical_features = ['OCC_DOY','OCC_DAY','OCC_HOUR']
label = ['OCC_MONTH']
target = ['OCC_MONTH']

In [141]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.25, random_state=42)

In [142]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [143]:
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),('numerical',numerical_transformer,numerical_features) ])

In [144]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [145]:
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_DOW',
                                                   'OCC_TIME_RANGE']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY',
                                                   'OCC_HOUR'])])),
                ('regressor', RandomForestClassifier())])

In [146]:
# Predict the probabilities of crime occurrence for each day of the week in the test set
day_probabilities = random_forest.predict_proba(X_test)

In [147]:
# Display the probabilities for each day of the week in the test set
day_names = label_encoder_trans.inverse_transform(random_forest.classes_)
probabilities_df = pd.DataFrame(day_probabilities, columns=day_names)
print(probabilities_df)

      April  August  December  February  January  July  June  March   May  \
0      0.00    0.29      0.06      0.00     0.00  0.02  0.01   0.02  0.01   
1      0.01    0.03      0.14      0.01     0.00  0.11  0.00   0.00  0.01   
2      0.08    0.00      0.00      0.02     0.68  0.06  0.01   0.15  0.00   
3      0.28    0.01      0.00      0.01     0.02  0.02  0.20   0.02  0.41   
4      0.00    0.65      0.00      0.02     0.00  0.17  0.03   0.00  0.01   
...     ...     ...       ...       ...      ...   ...   ...    ...   ...   
1422   0.02    0.00      0.01      0.18     0.78  0.00  0.01   0.00  0.00   
1423   0.00    0.02      0.47      0.00     0.00  0.00  0.00   0.01  0.00   
1424   0.00    0.02      0.67      0.01     0.00  0.01  0.01   0.00  0.01   
1425   0.00    0.01      0.00      0.17     0.00  0.00  0.06   0.08  0.68   
1426   0.00    0.00      0.00      0.00     0.00  0.02  0.03   0.00  0.00   

      November  October  September  
0         0.00     0.19       0.40  
1

In [148]:
predicted_classes = day_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to day names
predicted_day_names = label_encoder_trans.inverse_transform(predicted_classes)
print(predicted_day_names)
print(predicted_classes)

['September' 'October' 'January' ... 'December' 'May' 'October']
[11 10  4 ...  2  8 10]


In [149]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print("Accuracy:", accuracy)

Accuracy: 0.9446391030133147


In [150]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[110   0   0   4   0   1   0   9   8   0   0   0]
 [  0 136   0   0   0   0   0   0   0   0   0   0]
 [  0   0  90   0   0   0   0   0   0   4   0   0]
 [  1   0   0  64   2   0   0   8   0   0   0   0]
 [  0   0   0   1 101   0   0   0   0   0   0   0]
 [  0   1   0   0   0 146   1   0   0   0   0   0]
 [  0   0   0   0   0   2 127   0   2   0   0   0]
 [  9   0   0   6   0   0   0  84   2   0   0   0]
 [  6   0   0   0   0   0   3   0 125   0   0   0]
 [  0   0   6   0   0   0   0   0   0 106   1   0]
 [  0   0   0   0   0   0   0   0   0   0 108   1]
 [  0   0   0   0   0   0   0   0   0   0   1 151]]


In [151]:
# Calculate classification report (includes precision, recall, and F1-score for each class)
classification_rep = classification_report(y_test, predicted_classes, target_names=label_encoder_trans.classes_)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

       April       0.87      0.83      0.85       132
      August       0.99      1.00      1.00       136
    December       0.94      0.96      0.95        94
    February       0.85      0.85      0.85        75
     January       0.98      0.99      0.99       102
        July       0.98      0.99      0.98       148
        June       0.97      0.97      0.97       131
       March       0.83      0.83      0.83       101
         May       0.91      0.93      0.92       134
    November       0.96      0.94      0.95       113
     October       0.98      0.99      0.99       109
   September       0.99      0.99      0.99       152

    accuracy                           0.94      1427
   macro avg       0.94      0.94      0.94      1427
weighted avg       0.94      0.94      0.94      1427



In [152]:
# Calculate AUC-ROC (requires one-hot encoded y_test and day_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
auc_roc = roc_auc_score(y_test_one_hot, day_probabilities, multi_class='ovr')
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.9976577156196996


In [153]:
#get the day with the high mean of probability 
probabilities_df.describe(include='all')

,April,August,December,February,January,July,June,March,May,November,October,September
count,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000
mean,0.083518,0.097898,0.063560,0.061044,0.063973,0.105971,0.091114,0.075137,0.097239,0.075739,0.081955,0.102852
std,0.170086,0.221697,0.183289,0.138313,0.179225,0.230194,0.191536,0.156382,0.195490,0.185127,0.183380,0.219954
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.010000,0.020000,0.000000,0.000000,0.000000,0.020000,0.020000,0.010000,0.010000,0.000000,0.010000,0.010000
75%,0.070000,0.050000,0.020000,0.040000,0.020000,0.060000,0.065000,0.060000,0.080000,0.030000,0.060000,0.070000
max,0.930000,0.990000,0.980000,0.960000,0.950000,1.000000,0.990000,0.920000,0.990000,0.980000,0.970000,0.970000


vunerable month: July ,sept 

In [154]:
#hour of the day  
crime_data['OCC_HOUR'] = label_encoder_trans.fit_transform(crime_data['OCC_HOUR'])
target = crime_data['OCC_HOUR']

In [155]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_DOW','OCC_TIME_RANGE','OCC_MONTH']
numerical_features = ['OCC_DOY','OCC_DAY']
label = ['OCC_HOUR']
target = ['OCC_HOUR']

In [156]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.25, random_state=4567)

In [157]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [158]:
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),('numerical',numerical_transformer,numerical_features) ])

In [159]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [160]:
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_DOW', 'OCC_TIME_RANGE',
                                                   'OCC_MONTH']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY'])])),
                ('regressor', RandomForestClassifier())])

In [161]:
# Predict the probabilities of crime occurrence for each day of the week in the test set
day_probabilities = random_forest.predict_proba(X_test)

In [162]:
# Display the probabilities for each day of the week in the test set
day_names = label_encoder_trans.inverse_transform(random_forest.classes_)
probabilities_df = pd.DataFrame(day_probabilities, columns=day_names)
print(probabilities_df)

           0        1         2      3         4     5     6     7         8   \
0     0.00000  0.00000  0.000000  0.000  0.000000  0.00  0.00  0.00  0.000000   
1     0.17600  0.02000  0.129000  0.435  0.160000  0.07  0.00  0.00  0.000000   
2     0.00000  0.00000  0.000000  0.000  0.000000  0.00  0.39  0.02  0.023333   
3     0.27206  0.23931  0.420297  0.025  0.023333  0.02  0.00  0.00  0.000000   
4     0.07000  0.27500  0.300000  0.005  0.350000  0.00  0.00  0.00  0.000000   
...       ...      ...       ...    ...       ...   ...   ...   ...       ...   
1422  0.04000  0.29000  0.000000  0.630  0.010000  0.03  0.00  0.00  0.000000   
1423  0.00000  0.00000  0.000000  0.000  0.000000  0.00  0.00  0.00  0.000000   
1424  0.00000  0.00000  0.000000  0.000  0.000000  0.00  0.00  0.00  0.000000   
1425  0.00000  0.00000  0.000000  0.000  0.000000  0.00  0.00  0.00  0.000000   
1426  0.00000  0.00000  0.000000  0.000  0.000000  0.00  0.00  0.00  0.000000   

        9   ...   14   15  

In [163]:
predicted_classes = day_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to day names
predicted_day_names = label_encoder_trans.inverse_transform(predicted_classes)
print(predicted_day_names)
print(predicted_classes)

[22  3 11 ... 22 20 21]
[22  3 11 ... 22 20 21]


In [164]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print("Accuracy:", accuracy)

Accuracy: 0.20602662929222143


In [165]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[59 18 29 26 10  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [47 15 16 10 11  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [34 17 26 11 11  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [29 16 19 15  7  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [12  5 10  7  6  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [13  5  5  4  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  8  2  0  4  2  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  1  1  1  0  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  3  3  0  1  6  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  2  2  6  1  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  2  2  1  3  6  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  6  1  2  5  4  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  3  3  5  8  2  0  0  0  0  0  0]
 [ 0  0

In [166]:
# Calculate AUC-ROC (requires one-hot encoded y_test and day_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
auc_roc = roc_auc_score(y_test_one_hot, day_probabilities, multi_class='ovr')
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.8495016013148741


In [167]:
#get the day with the high mean of probability 
probabilities_df.describe(include='all')


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,...,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000,1427.000000
mean,0.117934,0.056849,0.072860,0.056572,0.040319,0.018949,0.013687,0.007638,0.007295,0.011102,...,0.029462,0.025944,0.029490,0.037572,0.048558,0.053053,0.057244,0.073947,0.078086,0.097402
std,0.228144,0.155438,0.171258,0.150282,0.127016,0.080562,0.072244,0.053799,0.050753,0.062267,...,0.109806,0.100535,0.109811,0.127570,0.141340,0.140967,0.146987,0.165100,0.173919,0.198296
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.100833,0.010000,0.030000,0.015000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.020000,0.040000,0.050000,0.066667
max,0.960000,0.953036,0.952500,0.890000,0.980000,0.700000,0.710000,0.880000,0.788000,0.670000,...,0.860000,0.880000,0.851167,0.910000,0.896667,0.890000,0.870000,0.963333,0.927500,0.940000
